# Task B3 — Solution (Simple Rule-based Agents, NumPy)

In [ ]:
import numpy as np
TASKS = [
    {'name':'sum small list','type':'sum','input': np.array([1,2,3]), 'approved': True},
    {'name':'flag highs','type':'threshold','input': np.array([0.3, 0.7, 0.2]), 'threshold': 0.5, 'approved': True},
    {'name':'sum needs approval','type':'sum','input': np.array([5,5]), 'approved': False}
]
MAX_STEPS = 2
ALLOWED_TOOLS = {'sum_array','threshold_flag'}

In [ ]:
def sum_array(arr: np.ndarray):
    return float(np.sum(arr))
def threshold_flag(arr: np.ndarray, threshold: float=0.5):
    return (arr >= threshold).astype(int)

In [ ]:
def planner(task):
    t = task.get('type')
    if t == 'sum':
        return [{'tool':'sum_array','args':{'arr': task['input']}}]
    if t == 'threshold':
        return [{'tool':'threshold_flag','args':{'arr': task['input'], 'threshold': task.get('threshold',0.5)}}]
    return []

In [ ]:
class SimpleAgent:
    def __init__(self, allowed_tools, max_steps):
        self.allowed_tools = allowed_tools
        self.max_steps = max_steps
        self.logs = []
    def run(self, task):
        if not task.get('approved', False):
            self.logs.append({'error':'needs_approval'})
            return None
        plan = planner(task)
        if len(plan) == 0:
            self.logs.append({'error':'no_plan'})
            return None
        steps = 0
        last_out = None
        for step in plan:
            steps += 1
            if steps > self.max_steps:
                self.logs.append({'error':'max_steps_exceeded'})
                return None
            tool = step['tool']
            if tool not in self.allowed_tools:
                self.logs.append({'error':'tool_not_allowed', 'tool':tool})
                return None
            args = {'arr_shape': step['args']['arr'].shape}
            fn_args = step['args']
            out = sum_array(fn_args['arr']) if tool=='sum_array' else threshold_flag(fn_args['arr'], fn_args.get('threshold',0.5))
            self.logs.append({'tool':tool,'args_meta':args,'out_preview': str(type(out))})
            last_out = out
        return last_out

In [ ]:
agent = SimpleAgent(ALLOWED_TOOLS, MAX_STEPS)
outs = [agent.run(t) for t in TASKS]
print('outs:', outs)
assert isinstance(outs[0], float)
assert np.array_equal(outs[1], np.array([0,1,0]))
assert outs[2] is None
print('Solution checks passed.')